In [1]:
import numpy as np
import pandas as pd

import glam

%load_ext autoreload
%autoreload 2

# hierarchical models
## one factor
### one factor between subject

In [2]:
np.random.seed(1)

m1 = glam.GLAM()

groups = ['high', 'low']

N = dict(high=5, low=3)

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))

for group in groups:
    
    parameters = dict(v=np.clip(np.random.normal(loc=0.00007, scale=0.00001, size=N[group]), 0.00003, 0.00015),
                      s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N[group]), 0.005, 0.011),
                      gamma=np.clip(np.random.normal(loc=gamma[group]['mu'], scale=gamma[group]['sd'], size=N[group]), -1, 1),
                      tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N[group]), 0.2, 2),
                      t0=np.zeros(N[group]))
    
    m1.simulate_group(kind='individual',
                      parameters=parameters,
                      n_individuals=N[group],
                      n_trials=n_trials,
                      n_items=n_items,
                      label=group)
m1.data.rename(columns=dict(condition='group'), inplace=True)

m1.make_model(kind='hierarchical', 
              depends_on=dict(gamma='group'),
              t0_val=0)
m1.fit(method='ADVI', n_advi=10000)

Generating hierarchical model for 8 subjects...
Fitting 1 model(s) using ADVI...
  Fitting model 1 of 1...


Average Loss = 3,786.6: 100%|██████████| 10000/10000 [00:22<00:00, 444.86it/s]
Finished [100%]: Average Loss = 3,786.1


/!\ Automatically setting parameter precision...


In [3]:
estimates = glam.utils.get_estimates(m1)
estimates

/!\ Automatically setting parameter precision...


,group,v,gamma,s,tau,t0,subject
0,high,0.000063,-0.35,0.008077,1.02,0.0,0
1,low,0.000063,NaN,0.008077,1.02,0.0,0
2,high,0.000066,-0.58,0.010274,1.18,0.0,1
3,low,0.000066,NaN,0.010274,1.18,0.0,1
4,high,0.000054,-0.40,0.009200,1.42,0.0,2
5,low,0.000054,NaN,0.009200,1.42,0.0,2
6,high,0.000055,-0.50,0.011642,1.22,0.0,3
7,low,0.000055,NaN,0.011642,1.22,0.0,3
8,high,0.000063,-0.40,0.010327,1.60,0.0,4
9,low,0.000063,NaN,0.010327,1.60,0.0,4


### one factor within subject

In [4]:
np.random.seed(1)

m2 = glam.GLAM()

groups = ['high', 'low']

N = 4

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))

for group in groups:
    
    parameters = dict(v=np.clip(np.random.normal(loc=0.00007, scale=0.00001, size=N), 0.00003, 0.00015),
                      s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                      gamma=np.clip(np.random.normal(loc=gamma[group]['mu'], scale=gamma[group]['sd'], size=N), -1, 1),
                      tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                      t0=np.zeros(N))
    
    m2.simulate_group(kind='individual',
                      parameters=parameters,
                      n_individuals=N,
                      individual_idx=np.arange(N),
                      n_trials=n_trials,
                      n_items=n_items,
                      label=group)
m2.data.rename(columns=dict(condition='group'), inplace=True)

m2.make_model(kind='hierarchical', 
              depends_on=dict(gamma='group'),
              t0_val=0)
m2.fit(method='ADVI', n_advi=10000)

Generating hierarchical model for 4 subjects...
Fitting 1 model(s) using ADVI...
  Fitting model 1 of 1...


Average Loss = 3,761.9: 100%|██████████| 10000/10000 [00:21<00:00, 464.27it/s]
Finished [100%]: Average Loss = 3,762


/!\ Automatically setting parameter precision...


In [5]:
estimates = glam.utils.get_estimates(m2)
estimates

/!\ Automatically setting parameter precision...


,group,v,gamma,s,tau,t0,subject
0,high,0.000054,-0.39,0.009769,1.06,0.0,0
1,low,0.000054,0.29,0.009769,1.06,0.0,0
2,high,0.000043,-0.58,0.009136,1.49,0.0,1
3,low,0.000043,0.42,0.009136,1.49,0.0,1
4,high,0.000052,-0.44,0.011821,1.20,0.0,2
5,low,0.000052,0.50,0.011821,1.20,0.0,2
6,high,0.000059,-0.77,0.008839,1.46,0.0,3
7,low,0.000059,0.43,0.008839,1.46,0.0,3


## two factors
### two factors between subject

In [6]:
np.random.seed(1)

m3 = glam.GLAM()

N = 5

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))
v = dict(slow=dict(mu=0.00005, sd=0.00001),
         fast=dict(mu=0.00010, sd=0.00001))

for bias in ['high', 'low']:
    for speed in ['slow', 'fast']:
    
        parameters = dict(v=np.clip(np.random.normal(loc=v[speed]['mu'], scale=v[speed]['sd'], size=N), 0.00003, 0.00015),
                          s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                          gamma=np.clip(np.random.normal(loc=gamma[bias]['mu'], scale=gamma[bias]['sd'], size=N), -1, 1),
                          tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                          t0=np.zeros(N))

        m3.simulate_group(kind='individual',
                          parameters=parameters,
                          n_individuals=N,
                          n_trials=n_trials,
                          n_items=n_items,
                          label=bias + '_' + speed)

m3.data['bias'], m3.data['speed'] = m3.data['condition'].str.split('_', 1).str

m3.make_model(kind='hierarchical', 
              depends_on=dict(gamma='bias',
                              v='speed'),
              t0_val=0)
m3.fit(method='ADVI', n_advi=10000)

Generating hierarchical model for 20 subjects...
Fitting 1 model(s) using ADVI...
  Fitting model 1 of 1...


Average Loss = 9,256.9: 100%|██████████| 10000/10000 [00:39<00:00, 255.79it/s]
Finished [100%]: Average Loss = 9,255.7


/!\ Automatically setting parameter precision...


In [7]:
estimates = glam.utils.get_estimates(m3)
estimates.head()

/!\ Automatically setting parameter precision...


,bias,speed,v,gamma,s,tau,t0,subject
0,high,slow,0.00006,-0.42,0.010353,1.03,0.0,0
1,high,fast,NaN,-0.42,0.010353,1.03,0.0,0
2,low,slow,0.00006,NaN,0.010353,1.03,0.0,0
3,low,fast,NaN,NaN,0.010353,1.03,0.0,0
4,high,slow,0.00005,-0.56,0.011474,1.46,0.0,1


In [8]:
estimates.dropna().reset_index(drop=True).head()

,bias,speed,v,gamma,s,tau,t0,subject
0,high,slow,0.000060,-0.42,0.010353,1.03,0.0,0
1,high,slow,0.000050,-0.56,0.011474,1.46,0.0,1
2,high,slow,0.000032,-0.51,0.011495,1.08,0.0,2
3,high,slow,0.000039,-0.61,0.011378,1.02,0.0,3
4,high,slow,0.000047,-0.45,0.010549,1.29,0.0,4


### two factors within subject

In [9]:
np.random.seed(1)

m4 = glam.GLAM()

N = 5

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))
v = dict(slow=dict(mu=0.00005, sd=0.00001),
         fast=dict(mu=0.00010, sd=0.00001))

for bias in ['high', 'low']:
    for speed in ['slow', 'fast']:
    
        parameters = dict(v=np.clip(np.random.normal(loc=v[speed]['mu'], scale=v[speed]['sd'], size=N), 0.00003, 0.00015),
                          s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                          gamma=np.clip(np.random.normal(loc=gamma[bias]['mu'], scale=gamma[bias]['sd'], size=N), -1, 1),
                          tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                          t0=np.zeros(N))

        m4.simulate_group(kind='individual',
                          parameters=parameters,
                          individual_idx=np.arange(N),
                          n_individuals=N,
                          n_trials=n_trials,
                          n_items=n_items,
                          label=bias + '_' + speed)

m4.data['bias'], m4.data['speed'] = m4.data['condition'].str.split('_', 1).str

m4.make_model(kind='hierarchical', 
              depends_on=dict(gamma='bias',
                              v='speed'),
              t0_val=0)
m4.fit(method='ADVI', n_advi=10000)

Generating hierarchical model for 5 subjects...
Fitting 1 model(s) using ADVI...
  Fitting model 1 of 1...


Average Loss = 9,187.7: 100%|██████████| 10000/10000 [00:38<00:00, 258.62it/s]
Finished [100%]: Average Loss = 9,186


/!\ Automatically setting parameter precision...


In [10]:
estimates = glam.utils.get_estimates(m4)
estimates.head()

/!\ Automatically setting parameter precision...


,bias,speed,v,gamma,s,tau,t0,subject
0,high,slow,0.000055,-0.39,0.009473,1.16,0.0,0
1,high,fast,0.000073,-0.39,0.009473,1.16,0.0,0
2,low,slow,0.000055,0.68,0.009473,1.16,0.0,0
3,low,fast,0.000073,0.68,0.009473,1.16,0.0,0
4,high,slow,0.000043,-0.43,0.009083,1.09,0.0,1


# individual models
## one factor
### one factor between subject

In [44]:
np.random.seed(110101)

m5 = glam.GLAM()

groups = ['high', 'low']

N = dict(high=2, low=2)

n_trials = 50
n_items = 4

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))

for group in groups:

    parameters = dict(v=np.clip(np.random.normal(loc=0.00012, scale=0.00001, size=N[group]), 0.00003, 0.00015),
                      s=np.clip(np.random.normal(loc=0.01, scale=0.001, size=N[group]), 0.005, 0.011),
                      gamma=np.clip(np.random.normal(loc=gamma[group]['mu'], scale=gamma[group]['sd'], size=N[group]), -1, 1),
                      tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N[group]), 0.2, 2),
                      t0=np.zeros(N[group]))
    
    m5.simulate_group(kind='individual',
                      parameters=parameters,
                      n_individuals=N[group],
                      n_trials=n_trials,
                      n_items=n_items,
                      label=group)
m5.data.rename(columns=dict(condition='group'), inplace=True)

m5.make_model(kind='individual', 
              depends_on=dict(gamma='group'),
              t0_val=0)
m5.fit(method='NUTS', n_samples=1000, chains=2, njobs=1)

Generating single subject models for 4 subjects...


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Fitting 4 model(s) using NUTS...
  Fitting model 1 of 4...


Sequential sampling (2 chains in 1 job)
NUTS: [tau_interval__, s_interval__, gamma_interval__, v_interval__]
100%|██████████| 1500/1500 [00:05<00:00, 274.64it/s]
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 2 of 4...


...

  Fitting model 3 of 4...


Sequential sampling (2 chains in 1 job)
NUTS: [tau_interval__, s_interval__, gamma_interval__, v_interval__]
100%|██████████| 1500/1500 [00:10<00:00, 146.95it/s]
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There were 7 divergences after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 4 of 4...


Sequential sampling (2 chains in 1 job)
NUTS: [tau_interval__, s_interval__, gamma_interval__, v_interval__]
100%|██████████| 1500/1500 [00:07<00:00, 201.78it/s]
The acceptance probability does not match the target. It is 0.899872000437, but should be close to 0.8. Try to increase the number of tuning steps.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


In [47]:
estimates = glam.utils.get_estimates(m5)
estimates

/!\ Automatically setting parameter precision...


,gamma,group,s,subject,t0,tau,v
0,0.03,high,0.007991,0,0.0,0.93,0.000108
1,NaN,low,0.007991,0,0.0,0.93,0.000108
2,-0.15,high,0.011539,1,0.0,0.71,0.000139
3,NaN,low,0.011539,1,0.0,0.71,0.000139
4,NaN,high,0.008711,2,0.0,0.88,0.000139
5,0.62,low,0.008711,2,0.0,0.88,0.000139
6,NaN,high,0.006164,3,0.0,0.45,0.000121
7,0.66,low,0.006164,3,0.0,0.45,0.000121


In [48]:
estimates.dropna()

,gamma,group,s,subject,t0,tau,v
0,0.03,high,0.007991,0,0.0,0.93,0.000108
2,-0.15,high,0.011539,1,0.0,0.71,0.000139
5,0.62,low,0.008711,2,0.0,0.88,0.000139
7,0.66,low,0.006164,3,0.0,0.45,0.000121


### one factor within subject

In [51]:
np.random.seed(10101)

m6 = glam.GLAM()

groups = ['high', 'low']

N = 2

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))

for group in groups:
    
    parameters = dict(v=np.clip(np.random.normal(loc=0.00007, scale=0.00001, size=N), 0.00003, 0.00015),
                      s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                      gamma=np.clip(np.random.normal(loc=gamma[group]['mu'], scale=gamma[group]['sd'], size=N), -1, 1),
                      tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                      t0=np.zeros(N))
    
    m6.simulate_group(kind='individual',
                      parameters=parameters,
                      n_individuals=N,
                      individual_idx=np.arange(N),
                      n_trials=n_trials,
                      n_items=n_items,
                      label=group)
m6.data.rename(columns=dict(condition='group'), inplace=True)

m6.make_model(kind='individual', 
              depends_on=dict(gamma='group'),
              t0_val=0)
m6.fit(method='NUTS', n_samples=1000, chains=2, njobs=1)

Generating single subject models for 2 subjects...


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Fitting 2 model(s) using NUTS...
  Fitting model 1 of 2...


Sequential sampling (2 chains in 1 job)
NUTS: [tau_interval__, s_interval__, gamma_low_interval__, gamma_high_interval__, v_interval__]
100%|██████████| 1500/1500 [00:09<00:00, 165.27it/s]
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 2 of 2...


Sequential sampling (2 chains in 1 job)
NUTS: [tau_interval__, s_interval__, gamma_low_interval__, gamma_high_interval__, v_interval__]
100%|██████████| 1500/1500 [00:10<00:00, 146.46it/s]
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


In [52]:
estimates = glam.utils.get_estimates(m6)
estimates

/!\ Automatically setting parameter precision...


,gamma,group,s,subject,t0,tau,v
0,-0.41,high,0.008118,0,0.0,1.20,0.000068
1,0.34,low,0.008118,0,0.0,1.20,0.000068
2,-0.36,high,0.007106,1,0.0,0.96,0.000059
3,0.49,low,0.007106,1,0.0,0.96,0.000059


## two factors
### two factors between subject

In [56]:
np.random.seed(101110)

m7 = glam.GLAM()

N = 1

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))
v = dict(slow=dict(mu=0.00005, sd=0.00001),
         fast=dict(mu=0.00010, sd=0.00001))

for bias in ['high', 'low']:
    for speed in ['slow', 'fast']:
    
        parameters = dict(v=np.clip(np.random.normal(loc=v[speed]['mu'], scale=v[speed]['sd'], size=N), 0.00003, 0.00015),
                          s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                          gamma=np.clip(np.random.normal(loc=gamma[bias]['mu'], scale=gamma[bias]['sd'], size=N), -1, 1),
                          tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                          t0=np.zeros(N))

        m7.simulate_group(kind='individual',
                          parameters=parameters,
                          n_individuals=N,
                          n_trials=n_trials,
                          n_items=n_items,
                          label=bias + '_' + speed)

m7.data['bias'], m7.data['speed'] = m7.data['condition'].str.split('_', 1).str

m7.make_model(kind='individual', 
              depends_on=dict(gamma='bias',
                              v='speed'),
              t0_val=0)
m7.fit(method='NUTS', n_samples=1000, chains=2, njobs=1)

Generating single subject models for 4 subjects...


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Fitting 4 model(s) using NUTS...
  Fitting model 1 of 4...


Sequential sampling (2 chains in 1 job)
NUTS: [tau_interval__, s_interval__, gamma_interval__, v_interval__]
100%|██████████| 1500/1500 [00:06<00:00, 234.66it/s]
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 2 of 4...


...

  Fitting model 3 of 4...


Sequential sampling (2 chains in 1 job)
NUTS: [tau_interval__, s_interval__, gamma_interval__, v_interval__]
100%|██████████| 1500/1500 [00:13<00:00, 111.92it/s]
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 4 of 4...


Sequential sampling (2 chains in 1 job)
NUTS: [tau_interval__, s_interval__, gamma_interval__, v_interval__]
100%|██████████| 1500/1500 [00:07<00:00, 207.14it/s]
There were 14 divergences after tuning. Increase `target_accept` or reparameterize.
There were 16 divergences after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


In [57]:
estimates = glam.utils.get_estimates(m7)
estimates

/!\ Automatically setting parameter precision...


,bias,gamma,s,speed,subject,t0,tau,v
0,high,-0.24,0.008346,slow,0,0.0,1.08,0.000051
1,high,-0.24,0.008346,fast,0,0.0,1.08,NaN
2,low,NaN,0.008346,slow,0,0.0,1.08,0.000051
3,low,NaN,0.008346,fast,0,0.0,1.08,NaN
4,high,-0.37,0.010004,slow,1,0.0,1.06,NaN
5,high,-0.37,0.010004,fast,1,0.0,1.06,0.000096
6,low,NaN,0.010004,slow,1,0.0,1.06,NaN
7,low,NaN,0.010004,fast,1,0.0,1.06,0.000096
8,high,NaN,0.005481,slow,2,0.0,0.78,0.000033
9,high,NaN,0.005481,fast,2,0.0,0.78,NaN


### two factors within subject

In [61]:
np.random.seed(1011)

m8 = glam.GLAM()

N = 4

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))
v = dict(slow=dict(mu=0.00005, sd=0.00001),
         fast=dict(mu=0.00010, sd=0.00001))

for bias in ['high', 'low']:
    for speed in ['slow', 'fast']:
    
        parameters = dict(v=np.clip(np.random.normal(loc=v[speed]['mu'], scale=v[speed]['sd'], size=N), 0.00003, 0.00015),
                          s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                          gamma=np.clip(np.random.normal(loc=gamma[bias]['mu'], scale=gamma[bias]['sd'], size=N), -1, 1),
                          tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                          t0=np.zeros(N))

        m8.simulate_group(kind='individual',
                          parameters=parameters,
                          individual_idx=np.arange(N),
                          n_individuals=N,
                          n_trials=n_trials,
                          n_items=n_items,
                          label=bias + '_' + speed)

m8.data['bias'], m8.data['speed'] = m8.data['condition'].str.split('_', 1).str

m8.make_model(kind='individual', 
              depends_on=dict(gamma='bias',
                              v='speed'),
              t0_val=0)
m8.fit(method='NUTS', n_samples=1000, chains=2, njobs=1)

Generating single subject models for 4 subjects...


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Fitting 4 model(s) using NUTS...
  Fitting model 1 of 4...


Sequential sampling (2 chains in 1 job)
NUTS: [tau_interval__, s_interval__, gamma_low_interval__, gamma_high_interval__, v_fast_interval__, v_slow_interval__]
100%|██████████| 1500/1500 [00:11<00:00, 130.69it/s]
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 2 of 4...


...

  Fitting model 3 of 4...


Sequential sampling (2 chains in 1 job)
NUTS: [tau_interval__, s_interval__, gamma_low_interval__, gamma_high_interval__, v_fast_interval__, v_slow_interval__]
100%|██████████| 1500/1500 [00:12<00:00, 118.97it/s]
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 4 of 4...


Sequential sampling (2 chains in 1 job)
NUTS: [tau_interval__, s_interval__, gamma_low_interval__, gamma_high_interval__, v_fast_interval__, v_slow_interval__]
100%|██████████| 1500/1500 [00:16<00:00, 93.37it/s]
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


In [62]:
estimates = glam.utils.get_estimates(m8)
estimates

/!\ Automatically setting parameter precision...


,bias,gamma,s,speed,subject,t0,tau,v
0,high,-0.33,0.008095,slow,0,0.0,1.07,0.000053
1,high,-0.33,0.008095,fast,0,0.0,1.07,0.000082
2,low,0.46,0.008095,slow,0,0.0,1.07,0.000053
3,low,0.46,0.008095,fast,0,0.0,1.07,0.000082
4,high,-0.30,0.008682,slow,1,0.0,0.88,0.000059
5,high,-0.30,0.008682,fast,1,0.0,0.88,0.000111
6,low,0.48,0.008682,slow,1,0.0,0.88,0.000059
7,low,0.48,0.008682,fast,1,0.0,0.88,0.000111
8,high,-0.33,0.008841,slow,2,0.0,0.96,0.000049
9,high,-0.33,0.008841,fast,2,0.0,0.96,0.000098
